In [ ]:
import os
import shutil

import numpy as np
import re
import nltk
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import time

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
tf.get_logger().setLevel('ERROR')
seed = 42

# Utils

In [ ]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    # text = text.lower() # lowercase text
    # REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@;]')
    # BAD_SYMBOLS_RE = re.compile('[^0-9a-z#+_]')
    STOPWORDS = set(nltk.corpus.stopwords.words('english'))
    UNWANTED_WORDS = ['copyrights', 'copyright', 'results', 'objectives', 'methods', 'design', 'study population', 'measurements', 'conclusions', 'materials', 'methods', 'limitations', 'setting', 'purpose', 'intervention', 'main outcome', 'measures', 'background']
    # text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    # text = BAD_SYMBOLS_RE.sub(' ', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS and word not in UNWANTED_WORDS) # remove stopwords and UNWANTED_WORDS from text
    return text

In [ ]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text  # or whatever

In [ ]:
def preprocess_test(text):
  nan_a = text[text['Abstract'].isnull()]
  bool_pred = pd.isnull(text['Abstract'])
  for i, row in text[bool_pred].iterrows():
    text.at[i,'abstract'] = " "
  text = text.dropna(subset=["Abstract"])
  rows, cols = text.shape
  print('Removed nan from abstract: Data size: Num of rows: ',rows, ' Num of cols: ',cols)
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
  BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
  STOPWORDS = set(nltk.corpus.stopwords.words('english'))
  UNWANTED_WORDS = ['results', 'objectives', 'methods', 'design', 'study population', 'measurements', 'conclusions', 'materials', 'methods', 'limitations', 'setting', 'patients', 'purpose', 'intervention', 'main outcome', 'measures', 'background']
  text['TitlePred'] = text['Title'].apply(clean_text)
  text['AbstractPred'] = text['Abstract'].apply(clean_text)
  text['TitlePred'] = text['TitlePred'].str.replace('\d+', '')
  text['AbstractPred'] = text['AbstractPred'].str.replace('\d+', '')
  return text, nan_a

# Preprocess Data


In [ ]:
articles = pd.read_csv('/content/gdrive/MyDrive/ESA/sex_diff_screenings.csv')
test = pd.read_csv('/content/gdrive/MyDrive/ESA/test.csv')

In [ ]:
articles.shape

(3999, 23)

In [ ]:
test.shape

(1000, 22)

In [ ]:
test['Decision'] = test['ScreeningDecisions']

In [ ]:
# articles_merged = articles.append(test, ignore_index=True)

In [ ]:
# articles_merged['Decision']

0       Excluded
1       Excluded
2       Excluded
3       Included
4       Excluded
          ...   
5835    Excluded
5836    Excluded
5837    Excluded
5838    Excluded
5839    Excluded
Name: Decision, Length: 5840, dtype: object

In [ ]:
# articles_merged, nan = preprocess_test(articles_merged);

Removed nan from abstract: Data size: Num of rows:  5239  Num of cols:  34


<ipython-input-8-65ffb8115e33>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['TitlePred'] = text['Title'].apply(clean_text)
<ipython-input-8-65ffb8115e33>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['AbstractPred'] = text['Abstract'].apply(clean_text)
<ipython-input-8-65ffb8115e33>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  text['TitlePred'] = text['TitlePred'].str.replace('\d+', '')
<ipython-input-8-65ffb8115e33>:15: SettingWithCo

In [ ]:
articles, nan = preprocess_test(articles);
test, nan = preprocess_test(test);

Removed nan from abstract: Data size: Num of rows:  3631  Num of cols:  24


<ipython-input-8-65ffb8115e33>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['TitlePred'] = text['Title'].apply(clean_text)
<ipython-input-8-65ffb8115e33>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['AbstractPred'] = text['Abstract'].apply(clean_text)
<ipython-input-8-65ffb8115e33>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  text['TitlePred'] = text['TitlePred'].str.replace('\d+', '')
<ipython-input-8-65ffb8115e33>:15: SettingWithCo

Removed nan from abstract: Data size: Num of rows:  1000  Num of cols:  23


In [ ]:
articles = articles.drop_duplicates(subset=["TitlePred"])
articles = articles.drop_duplicates(subset=['AbstractPred'])
test = test.drop_duplicates(subset=["TitlePred"])
test = test.drop_duplicates(subset=['AbstractPred'])

In [ ]:
articles["titleabstract"] = articles["TitlePred"] + " " + articles["AbstractPred"]

In [ ]:
test["titleabstract"] = test["TitlePred"] + " " + test["AbstractPred"]

In [ ]:
X = articles['titleabstract'].values
Y = articles['Decision'].values

In [ ]:
titles = articles['TitlePred'].values
abstracts = articles['AbstractPred'].values
Y = articles['Decision'].values

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 2021)

In [ ]:
nr_train = int(np.ceil(titles.shape[0]*0.9))
# nr_val = int(np.ceil(titles.shape[0]*0.9))

In [ ]:
titles_train = titles[0:nr_train]
abstracts_train = abstracts[0:nr_train]
Y_train = Y[0:nr_train]

In [ ]:
titles_val = titles[nr_train:]
abstracts_val = abstracts[nr_train:]
Y_val = Y[nr_train:]

In [ ]:
titles_test = titles[nr_val:]
abstracts_test = abstracts[nr_val:]
Y_test = Y[nr_val:]

In [ ]:
train_data = {"titles": titles_train, "abstracts": abstracts_train, 'decision': Y_train}
train_df = pd.DataFrame(data=train_data)

In [ ]:
train_df['titleabstract'] = train_df['titles'] + '. ' + train_df['abstracts']

In [ ]:
val_data = {"titles": titles_val, "abstracts": abstracts_val, 'decision': Y_val}
val_df = pd.DataFrame(data=val_data)

In [ ]:
val_df['titleabstract'] = val_df['titles'] + '. ' + val_df['abstracts']

In [ ]:
test_data = {"titles": titles_test, "abstracts": abstracts_test, 'decision': Y_test}
test_df = pd.DataFrame(data=test_data)

In [ ]:
test_df['titleabstract'] = test_df['titles'] + '. ' + test_df['abstracts']

In [ ]:
test_df[test_df.decision == 'Included'].shape[0] / test_df.shape[0]

In [ ]:
val_df[val_df.decision == 'Included'].shape[0] / val_df.shape[0]

0.04709141274238227

In [ ]:
train_df[train_df.decision == 'Included'].shape[0] / train_df.shape[0]

0.06263432606693276

In [ ]:
train_df.to_csv("sex_diff_train.csv")

In [ ]:
val_df.to_csv("sex_diff_val.csv")

In [ ]:
test.to_csv("sex_diff_test.csv")